In [ ]:
# from https://keras.io/examples/structured_data/structured_data_classification_from_scratch/

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
#file_url = ""
dataframe = pd.read_csv('train.csv',index_col=0)

In [3]:
dataframe.shape

(900000, 32)

In [4]:
dataframe.head()

,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,f_09,...,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,target
id,,,,,,,,,,,,,,,,,,,,,
0,-1.373246,0.238887,-0.243376,0.567405,-0.647715,0.839326,0.113133,1,5,1,...,-2.540739,0.766952,-2.730628,-0.208177,1.363402,ABABDADBAB,67.609153,0,0,0
1,1.697021,-1.710322,-2.230332,-0.545661,1.113173,-1.552175,0.447825,1,3,4,...,2.278315,-0.633658,-1.217077,-3.782194,-0.058316,ACACCADCEB,377.096415,0,0,1
2,1.681726,0.616746,-1.027689,0.810492,-0.609086,0.113965,-0.708660,1,0,2,...,-1.385775,-0.520558,-0.009121,2.788536,-3.703488,AAAEABCKAD,-195.599702,0,2,1
3,-0.118172,-0.587835,-0.804638,2.086822,0.371005,-0.128831,-0.282575,3,2,1,...,0.572594,-1.653213,1.686035,-2.533098,-0.608601,BDBBAACBCB,210.826205,0,0,1
4,1.148481,-0.176567,-0.664871,-1.101343,0.467875,0.500117,0.407515,3,3,0,...,-3.912929,-1.430366,2.127649,-3.306784,4.371371,BDBCBBCHFE,-217.211798,0,1,1


In [5]:
val_dataframe = dataframe.sample(frac=0.2, random_state=1337)
train_dataframe = dataframe.drop(val_dataframe.index)

print(
    "Using %d samples for training and %d for validation"
    % (len(train_dataframe), len(val_dataframe))
)

Using 720000 samples for training and 180000 for validation


In [6]:
def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop("target")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds


train_ds = dataframe_to_dataset(train_dataframe)
val_ds = dataframe_to_dataset(val_dataframe)

In [7]:
for x, y in train_ds.take(1):
    print("Input:", x)
    print("Target:", y)

Input: {'f_00': <tf.Tensor: shape=(), dtype=float64, numpy=-0.5087815693703458>, 'f_01': <tf.Tensor: shape=(), dtype=float64, numpy=0.3387263964164611>, 'f_02': <tf.Tensor: shape=(), dtype=float64, numpy=1.8143203574878808>, 'f_03': <tf.Tensor: shape=(), dtype=float64, numpy=-0.9300237468381168>, 'f_04': <tf.Tensor: shape=(), dtype=float64, numpy=-0.1953168112966872>, 'f_05': <tf.Tensor: shape=(), dtype=float64, numpy=-2.055904172831412>, 'f_06': <tf.Tensor: shape=(), dtype=float64, numpy=-0.1333089618735877>, 'f_07': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'f_08': <tf.Tensor: shape=(), dtype=int64, numpy=3>, 'f_09': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'f_10': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'f_11': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'f_12': <tf.Tensor: shape=(), dtype=int64, numpy=4>, 'f_13': <tf.Tensor: shape=(), dtype=int64, numpy=3>, 'f_14': <tf.Tensor: shape=(), dtype=int64, numpy=3>, 'f_15': <tf.Tensor: shape=(), dtype=int64, numpy=5>, 'f_16':

In [8]:
train_ds = train_ds.batch(32)
val_ds = val_ds.batch(32)

In [9]:
from tensorflow.keras.layers import IntegerLookup
from tensorflow.keras.layers import Normalization
from tensorflow.keras.layers import StringLookup


def encode_numerical_feature(feature, name, dataset):
    # Create a Normalization layer for our feature
    normalizer = Normalization()

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the statistics of the data
    normalizer.adapt(feature_ds)

    # Normalize the input feature
    encoded_feature = normalizer(feature)
    return encoded_feature


def encode_categorical_feature(feature, name, dataset, is_string):
    lookup_class = StringLookup if is_string else IntegerLookup
    # Create a lookup layer which will turn strings into integer indices
    lookup = lookup_class(output_mode="binary")

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the set of possible string values and assign them a fixed integer index
    lookup.adapt(feature_ds)

    # Turn the string input into integer indices
    encoded_feature = lookup(feature)
    return encoded_feature

In [10]:
train_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 720000 entries, 0 to 899998
Data columns (total 32 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   f_00    720000 non-null  float64
 1   f_01    720000 non-null  float64
 2   f_02    720000 non-null  float64
 3   f_03    720000 non-null  float64
 4   f_04    720000 non-null  float64
 5   f_05    720000 non-null  float64
 6   f_06    720000 non-null  float64
 7   f_07    720000 non-null  int64  
 8   f_08    720000 non-null  int64  
 9   f_09    720000 non-null  int64  
 10  f_10    720000 non-null  int64  
 11  f_11    720000 non-null  int64  
 12  f_12    720000 non-null  int64  
 13  f_13    720000 non-null  int64  
 14  f_14    720000 non-null  int64  
 15  f_15    720000 non-null  int64  
 16  f_16    720000 non-null  int64  
 17  f_17    720000 non-null  int64  
 18  f_18    720000 non-null  int64  
 19  f_19    720000 non-null  float64
 20  f_20    720000 non-null  float64
 21  f_21    72

In [11]:
# Categorical features encoded as integers
f_07 = keras.Input(shape=(1,), name="f_07", dtype="int64")
f_08 = keras.Input(shape=(1,), name="f_08", dtype="int64")
f_09 = keras.Input(shape=(1,), name="f_09", dtype="int64")
f_10 = keras.Input(shape=(1,), name="f_10", dtype="int64")
f_11 = keras.Input(shape=(1,), name="f_11", dtype="int64")
f_12 = keras.Input(shape=(1,), name="f_12", dtype="int64")
f_13 = keras.Input(shape=(1,), name="f_13", dtype="int64")
f_14 = keras.Input(shape=(1,), name="f_14", dtype="int64")
f_15 = keras.Input(shape=(1,), name="f_15", dtype="int64")
f_16 = keras.Input(shape=(1,), name="f_16", dtype="int64")
f_17 = keras.Input(shape=(1,), name="f_17", dtype="int64")
f_18 = keras.Input(shape=(1,), name="f_18", dtype="int64")
f_29 = keras.Input(shape=(1,), name="f_29", dtype="int64")
f_30 = keras.Input(shape=(1,), name="f_30", dtype="int64")

# Categorical feature encoded as string
#thal = keras.Input(shape=(1,), name="thal", dtype="string")




# String categorical features
#thal_encoded = encode_categorical_feature(thal, "thal", train_ds, True)



In [12]:
# Numerical features
f_00 = keras.Input(shape=(1,), name="f_01")
f_01 = keras.Input(shape=(1,), name="f_01")
f_02 = keras.Input(shape=(1,), name="f_02")
f_03 = keras.Input(shape=(1,), name="f_03")
f_04 = keras.Input(shape=(1,), name="f_04")
f_05 = keras.Input(shape=(1,), name="f_05")
f_06 = keras.Input(shape=(1,), name="f_06")
f_19 = keras.Input(shape=(1,), name="f_19")
f_20 = keras.Input(shape=(1,), name="f_20")
f_21 = keras.Input(shape=(1,), name="f_21")
f_22 = keras.Input(shape=(1,), name="f_22")
f_23 = keras.Input(shape=(1,), name="f_23")
f_24 = keras.Input(shape=(1,), name="f_24")
f_25 = keras.Input(shape=(1,), name="f_25")
f_26 = keras.Input(shape=(1,), name="f_26")
f_28 = keras.Input(shape=(1,), name="f_28")


In [13]:
all_inputs = [
    f_00,    f_01,    f_02,    f_03,    f_04,    f_05,    f_06,    f_07,    f_08,    f_09,    f_10,
    f_11,    f_12,    f_13,    f_14,    f_15,    f_16,    f_17,    f_18,    f_19,    f_20,
    f_21,    f_22,    f_23,    f_24,    f_25,    f_26,    f_28,    f_29,    f_30,
]



In [14]:
# Integer categorical features
f_07_encoded = encode_categorical_feature(f_07, "f_07", train_ds, False)
f_08_encoded = encode_categorical_feature(f_08, "f_08", train_ds, False)
f_09_encoded = encode_categorical_feature(f_09, "f_09", train_ds, False)
f_10_encoded = encode_categorical_feature(f_10, "f_10", train_ds, False)
f_11_encoded = encode_categorical_feature(f_11, "f_11", train_ds, False)
f_12_encoded = encode_categorical_feature(f_12, "f_12", train_ds, False)
f_13_encoded = encode_categorical_feature(f_13, "f_13", train_ds, False)
f_14_encoded = encode_categorical_feature(f_14, "f_14", train_ds, False)
f_15_encoded = encode_categorical_feature(f_15, "f_15", train_ds, False)
f_16_encoded = encode_categorical_feature(f_16, "f_16", train_ds, False)
f_17_encoded = encode_categorical_feature(f_17, "f_17", train_ds, False)
f_18_encoded = encode_categorical_feature(f_18, "f_18", train_ds, False)
f_29_encoded = encode_categorical_feature(f_29, "f_29", train_ds, False)
f_30_encoded = encode_categorical_feature(f_30, "f_30", train_ds, False)


KeyboardInterrupt: 

In [ ]:
# Numerical features
f_00_encoded = encode_numerical_feature(f_00, "f_00", train_ds)
f_01_encoded = encode_numerical_feature(f_01, "f_01", train_ds)
f_02_encoded = encode_numerical_feature(f_02, "f_02", train_ds)
f_03_encoded = encode_numerical_feature(f_03, "f_03", train_ds)
f_04_encoded = encode_numerical_feature(f_04, "f_04", train_ds)
f_05_encoded = encode_numerical_feature(f_05, "f_05", train_ds)
f_06_encoded = encode_numerical_feature(f_06, "f_06", train_ds)
f_19_encoded = encode_numerical_feature(f_19, "f_19", train_ds)
f_20_encoded = encode_numerical_feature(f_20, "f_20", train_ds)
f_21_encoded = encode_numerical_feature(f_21, "f_21", train_ds)
f_22_encoded = encode_numerical_feature(f_22, "f_22", train_ds)
f_23_encoded = encode_numerical_feature(f_23, "f_23", train_ds)
f_24_encoded = encode_numerical_feature(f_24, "f_24", train_ds)
f_25_encoded = encode_numerical_feature(f_25, "f_25", train_ds)
f_26_encoded = encode_numerical_feature(f_26, "f_26", train_ds)
f_28_encoded = encode_numerical_feature(f_28, "f_28", train_ds)



In [ ]:
all_features = layers.concatenate(
    [
        f_00_encoded, f_01_encoded,f_02_encoded,f_03_encoded,f_04_encoded,f_05_encoded,f_06_encoded,f_07_encoded,f_08_encoded,f_09_encoded,
        f_10_encoded, f_11_encoded,f_12_encoded,f_13_encoded,f_14_encoded,f_15_encoded,f_16_encoded,f_17_encoded,f_18_encoded,f_19_encoded,
        f_20_encoded, f_21_encoded,f_22_encoded,f_23_encoded,f_24_encoded,f_25_encoded,f_26_encoded,f_28_encoded,f_29_encoded,f_30_encoded,
    ]
)


In [ ]:
x = layers.Dense(32, activation="relu")(all_features)
x = layers.Dropout(0.5)(x)
output = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(all_inputs, output)
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])

In [13]:
# `rankdir='LR'` is to make the graph horizontal.
keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [14]:
model.fit(train_ds, epochs=50, validation_data=val_ds)

Epoch 1/50
8/8 [==============================] - 7s 332ms/step - loss: 0.5478 - accuracy: 0.7562 - val_loss: 0.5066 - val_accuracy: 0.7541
Epoch 2/50
8/8 [==============================] - 0s 63ms/step - loss: 0.5624 - accuracy: 0.7231 - val_loss: 0.4743 - val_accuracy: 0.7541
Epoch 3/50
8/8 [==============================] - 1s 78ms/step - loss: 0.5033 - accuracy: 0.7645 - val_loss: 0.4503 - val_accuracy: 0.7869
Epoch 4/50
8/8 [==============================] - 0s 46ms/step - loss: 0.4465 - accuracy: 0.7769 - val_loss: 0.4295 - val_accuracy: 0.7869
Epoch 5/50
8/8 [==============================] - 1s 86ms/step - loss: 0.4478 - accuracy: 0.8099 - val_loss: 0.4126 - val_accuracy: 0.8033
Epoch 6/50
8/8 [==============================] - 0s 68ms/step - loss: 0.4440 - accuracy: 0.7851 - val_loss: 0.3991 - val_accuracy: 0.8033
Epoch 7/50
8/8 [==============================] - 1s 75ms/step - loss: 0.4013 - accuracy: 0.8347 - val_loss: 0.3890 - val_accuracy: 0.7869
Epoch 8/50
8/8 [==========

In [15]:
sample = {
    "age": 60,
    "sex": 1,
    "cp": 1,
    "trestbps": 145,
    "chol": 233,
    "fbs": 1,
    "restecg": 2,
    "thalach": 150,
    "exang": 0,
    "oldpeak": 2.3,
    "slope": 3,
    "ca": 0,
    "thal": "fixed",
}

input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}
predictions = model.predict(input_dict)

print(
    "This particular patient had a %.1f percent probability "
    "of having a heart disease, as evaluated by our model." % (100 * predictions[0][0],)
)

This particular patient had a 19.2 percent probability of having a heart disease, as evaluated by our model.
